In [1]:
#get_ipython().magic('reset -sf')
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
# from modules
import auxiliary as aux

from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}

curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


In [2]:
curva['t'] = curva['plazo'] / 365
curva['rate'] = np.log(1 / curva['df'])/(curva['plazo'] / 365)
curva.head().style.format(frmt)

,plazo,tasa,df,t,rate
0,1,0.0811%,99.999778%,0.002740,0.000811
1,7,0.0841%,99.998388%,0.019178,0.000841
2,14,0.0780%,99.997010%,0.038356,0.000780
3,21,0.0774%,99.995549%,0.057534,0.000774
4,33,0.0781%,99.992942%,0.090411,0.000781


In [3]:
zcurva = interp1d (curva['t'],
                  curva ['rate'],
                  kind = 'cubic',
                  fill_value="extrapolate")
def zrate (t):
    return zcurva(t)
def dzrate(t): 
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta
def d2zrate(t):
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta
def fwd(t) :
    return zrate(t) + t*dzrate(t)
def dfwd(t):
    return 2 * dzrate(t) + t * d2zrate(t)

In [4]:
def b_hw(gamma, t, T):
    aux = 1- math.exp(- gamma * (T - t))
    return aux / gamma

def a_hw(zrate, fwd, gamma, sigma, t, T,
         verbose = False):
    b = b_hw(gamma, t, T)
    dfT = math.exp (-zrate(T) * T)
    dft = math.exp (-zrate(t) * t)
    c1 = math.log (dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    return c1 + c2 - c3

In [5]:
def zero_hw(r, gamma, sigma,
            zrate, fwd, t, T):
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

def theta(t):
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):

    dt = 1 / 264.0
    num_steps += 1
    
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(1, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return  sim

In [6]:
def Call_Put(r, t, T0, TB, K, gamma, epsilon, sigma):
    b = b_hw(gamma, T0, TB)
    Sz = b * math.sqrt(((sigma**2)/(2*gamma))*(1-math.exp(-2*gamma*T0)))
    Z0 = zero_hw(r, gamma, sigma, zrate, fwd, t, T0)
    ZB = zero_hw(r, gamma, sigma, zrate, fwd, t, TB)
    c1 = math.log (ZB / (K*Z0))
    d1 = (c1 + 0.5 * (Sz**2)) / math.sqrt(Sz) # AD: tiene el error de la fórmula que se corrigió en clase.
    d2 = d1 - Sz
    Div = ZB / Z0
    Norm1 = norm.cdf(epsilon * d1)
    Norm2 = norm.cdf(epsilon * d2)
    Call_Put = Z0*(epsilon*(Div * Norm1 - K*Norm2))
    return Call_Put

#Depende de lo que se quiera, si se quiere una call se utiliza epsilon = 1 y si se quiere una put, un epsilon = -1.

In [7]:
def Opcion_Monte_Carlo (epsilon, K, zrate, gamma, sigma, theta, r0, num_sim, num_step, T0, TB, fwd, seed ):
    df = curva['df'].iloc[15]
    precios = []
    CallPut_sim = []
    sim = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_step, seed)
    for i in range (num_sim):
        r = sim[i][-1]
        precios.append(zero_hw(r, gamma, sigma, zrate, fwd, T0, TB))  
        CallPut_sim.append(max((epsilon*(precios[i] - K)),0))
    precio_CallPut = df * np.mean(CallPut_sim)   
    return precio_CallPut

In [8]:
K = 1.01
epsilon = 1
gamma = .5
sigma = .015
num_sim = 2000
num_step = 254
seed = 2
r0 = zrate(0)
t=0
T0 = 1
TB = 2
Teorico_Call_Put = Call_Put(r0, t, T0, TB, K, gamma, epsilon, sigma)
Monte_Carlo_CallPut = Opcion_Monte_Carlo (epsilon, K, zrate, gamma, sigma, theta, r0, num_sim, num_step, T0, TB, fwd, seed )

**AD:** para este strike da valores negativos.

In [9]:
#Ya que se uso un valor de epsilon = 1, se mostrara el error de una call
print (f'Valor teórico de una call: {Teorico_Call_Put: .6}, \n Valor por simulación de una call: {Monte_Carlo_CallPut: .6}, \n Error call: {abs(Monte_Carlo_CallPut - Teorico_Call_Put): .6}')

Valor teórico de una call: -0.0010142, 
 Valor por simulación de una call:  0.000636566, 
 Error call:  0.00165076


In [10]:
def Bono_0_Cupon_Monte_Carlo (zrate, gamma, sigma, theta, r, num_sim, num_step, T0, TB, fwd, seed ):
    values = 0
    sim = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_step, seed)
    dt = 1 / 264.0
    for i in sim:
        values += math.exp(-dt * np.sum(i))
        bono = values / num_sim
    return bono

Teorico_Bono = curva['df'].iloc[15]
Montecarlo_Bono = Bono_0_Cupon_Monte_Carlo (zrate, gamma, sigma, theta, r0, num_sim, num_step, T0, TB, fwd, seed )

In [11]:
print(f' Valor teórico bono cero cupon a 1Y: {Teorico_Bono: .6}, \n Valor por simulación del bono a 1Y: {Montecarlo_Bono: .6}, \n Error del bono a 1Y: { abs(Montecarlo_Bono-Teorico_Bono): .6}')

 Valor teórico bono cero cupon a 1Y:  0.999298, 
 Valor por simulación del bono a 1Y:  0.999379, 
 Error del bono a 1Y:  8.12949e-05


**AD:** la fórmula analítica está mal. No se analiza la Put. No hay explicaciones.